<a href="https://colab.research.google.com/github/gabrielcerono/GlioblastomaMultiforme/blob/main/Survival_Analysis_Lammer2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip
!pip uninstall --yes --quiet osqp
!pip install -U scikit-survival

     |████████████████████████████████| 1.6MB 11.3MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 2.4 MB 14.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 2.3 MB 61.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 212 kB 68.1 MB/s 
  Created wheel for scikit-survival: filename=scikit_survival-0.14.0-cp37-cp37m-linux_x86_64.whl size=4061841 sha256=fd6b2eb057bf5f055d3d906798a27ae90f72ddfc1072ea19d4592e7567fb7565
  Stored in directory: /root/.cache/pip/wheels/a2/3e/97/3722ba215d3dfe5429c1a7e4f24f535a3f46004fb29a16d505
Successfully built scikit-survival


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk

In [3]:
pip install eli5

     |████████████████████████████████| 106 kB 12.3 MB/s 


In [4]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored
from sksurv.metrics import brier_score

In [6]:
dataset = pd.read_excel('/content/glioma.xlsx')

In [7]:
dataset.head()

,patient ID,sex,age [years],OS [months],"OS status (1=deceased, 0=living)",PFS [months],"progression (1=yes, 0=no)","MGMT methylation (1=methylated, 0=unmethylated)",cHsp70
0,1,m,62,1.2,1,1.2,1,0,high
1,2,m,58,20.9,1,18.2,1,0,high
2,3,f,50,4.0,1,4.6,1,0,low
3,4,m,20,26.0,1,16.1,1,1,high
4,6,m,42,15.8,1,6.2,1,0,low


In [8]:
dataset = dataset.drop(columns=['patient ID'])

In [9]:
dataset.columns

Index(['sex', 'age [years]', 'OS [months]', 'OS status (1=deceased, 0=living)',
       'PFS [months]', 'progression (1=yes, 0=no)',
       'MGMT methylation (1=methylated, 0=unmethylated)', 'cHsp70'],
      dtype='object')

In [10]:
dataset = dataset.rename(columns={'age [years]' : 'age', 'OS [months]' : 'os', 'OS status (1=deceased, 0=living)' : 'death',
       'progression (1=yes, 0=no)' : "didprogress",'MGMT methylation (1=methylated, 0=unmethylated)' : 'mgmt_methylation', 'cHsp70' : 'chsp70_0h1l', 'PFS [months]': 'pfs'})

In [11]:
dataset.head()

,sex,age,os,death,pfs,didprogress,mgmt_methylation,chsp70_0h1l
0,m,62,1.2,1,1.2,1,0,high
1,m,58,20.9,1,18.2,1,0,high
2,f,50,4.0,1,4.6,1,0,low
3,m,20,26.0,1,16.1,1,1,high
4,m,42,15.8,1,6.2,1,0,low


In [12]:
from sklearn.preprocessing import LabelEncoder
from pandas import get_dummies

In [13]:
label_encoder = LabelEncoder()
dataset['sex'] = label_encoder.fit_transform(dataset['sex'])
dataset['chsp70_0h1l'] = label_encoder.fit_transform(dataset['chsp70_0h1l'])

Let's try to predict first overall survival

1.   We need to map death as True or False
2.   We need to put death event first, then the right censored data. Example : (True, 21 months) 
3.   Then we have to pass from pandas DataFrame to structured array
4.   We can recover the data frame from records




In [14]:
dataset.corr(method='pearson')

,sex,age,os,death,pfs,didprogress,mgmt_methylation,chsp70_0h1l
sex,1.000000,-0.269165,-0.016478,0.138013,0.039569,-0.010193,-0.168035,-0.058461
age,-0.269165,1.000000,0.061198,-0.080623,0.177742,-0.161440,0.118375,-0.079750
os,-0.016478,0.061198,1.000000,-0.185224,0.873120,-0.288235,0.330128,-0.313911
death,0.138013,-0.080623,-0.185224,1.000000,-0.223676,0.369274,-0.265897,0.127076
pfs,0.039569,0.177742,0.873120,-0.223676,1.000000,-0.456801,0.320183,-0.328388
didprogress,-0.010193,-0.161440,-0.288235,0.369274,-0.456801,1.000000,-0.010336,0.229416
mgmt_methylation,-0.168035,0.118375,0.330128,-0.265897,0.320183,-0.010336,1.000000,-0.386500
chsp70_0h1l,-0.058461,-0.079750,-0.313911,0.127076,-0.328388,0.229416,-0.386500,1.000000


In [22]:
Y = dataset[['death', 'os']]

In [ ]:
Y['death'] = Y['death'].map({1: True, 0: False})

In [24]:
Y = Y.to_records(index=False)

In [60]:
X = dataset.drop(columns = ['death', 'os'])

In [26]:
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import train_test_split


In [27]:
rsf = RandomSurvivalForest()

In [28]:
from eli5.sklearn import PermutationImportance
import eli5


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [29]:
from sksurv.metrics import (concordance_index_censored,
                            concordance_index_ipcw,
                            cumulative_dynamic_auc)

Let's build a function, to use a rank feature to permutate, and select feature importance and a test split to check for the C-score


In [46]:
def permutation(X, y, model, loops):
  rankings = np.zeros(len(X.columns),)
  c_score_total = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same test 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state = seed )
    X_train, X_rank, y_train, y_rank = train_test_split(
    X, Y, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, y_train)
  #Let's define the permutation.
    permuter = PermutationImportance(
    estimator = model,
    n_iter = 10)
  #Let's fit the permutator on the Y test, (This permutation only shuffle the Y test.)
    permuter.fit(X_rank, y_rank)
  
    columns = X_test.columns.to_list()
  #The feature importance will give us and n array of how much 
    feature_importance = permuter.feature_importances_
    rankings = np.add(feature_importance, rankings)
  #Let's compute the scores
    c_score = model.score(X_test, y_test)
    c_score_total.append(c_score)

  
    
  #Dividing to get the average 
  ranking_terminal = np.true_divide(rankings, loops) 
  c_rank = pd.Series(ranking_terminal, columns).sort_values(ascending = False)
  c_mean = np.mean(np.array(c_score_total))
  



  return c_rank, c_mean

Let's try a 1000 looping permutation 

In [61]:
rank, score = permutation(X, Y, rsf, 1000)

In [64]:
rank

pfs                 0.216566
mgmt_methylation    0.015568
chsp70_0h1l         0.007754
age                 0.002757
didprogress         0.001232
sex                -0.006497
dtype: float64

In [65]:
score

0.7539081090977212

Let's run again, but with the top 2 features using 

In [52]:
X_tf = dataset[['pfs', 'mgmt_methylation']]

In [55]:
def top2features(X, y, model, loops):
  
  c_score_total = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same array of test sets, that we had in the previous feature selection algorithm. 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state = seed )
    X_train, X_rank, y_train, y_rank = train_test_split(
    X, Y, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, y_train)
  #Let's compute the scores
    c_score = model.score(X_test, y_test)
    c_score_total.append(c_score)

  
  c_mean_top2 = np.mean(np.array(c_score_total))
  



  return c_mean_top2

In [58]:
c_score_tf = top2features(X_tf, Y, rsf, 1000)

Prediction using only the top 2 features


In [59]:
c_score_tf

0.7685409921681158